# Analysis Notebook - Hierarchical Bayesian Modelling sex-as events

## **NOTE**:

We assume that you have cloned the analysis repository and have `cd` into the parent directory. Before starting with the analysis make sure you have first completed the dependencies set up by following the instructions described in the **`dependencies/README.md`** document. All paths defined in this Notebook are relative to the parent directory (repository). Please close this Notebook and start again by following the above guidelines if you have not completed the aforementioned steps.

## Prerequisite input files

Before starting the execution of the following code, make sure you have available in the folders `sbas/data` and `sbas/assets` the files listed below as prerequisites.

###  **`sbas/data`**.
The present analysis requires the following files to be present in the folder **`sbas/data`** uncompressed as the filenames indicate below:

- [x] `fromGTF.SE.txt`
- [x] `GTEx_Analysis_2017-06-05_v8_RNASeQCv1.1.9_gene_tpm.gct`
- [x] `rmats_final.a3ss.jc.ijc.txt`
- [x] `rmats_final.a3ss.jc.sjc.txt`
- [x] `rmats_final.a5ss.jc.ijc.txt`
- [x] `rmats_final.a5ss.jc.sjc.txt`
- [x] `rmats_final.mxe.jc.ijc.txt`
- [x] `rmats_final.mxe.jc.sjc.txt`
- [x] `rmats_final.ri.jc.ijc.txt`
- [x] `rmats_final.ri.jc.sjc.txt`
- [x] `rmats_final.se.jc.ijc.txt`
- [x] `rmats_final.se.jc.sjc.txt`
- [x] `*_model_B_sex_as_events*`


### **`sbas/assets`**
The present analysis requires the following files to be present in the folder **`sbas/assets`**.

- [x] `tissues.tsv`: metadata file with information on which tissues will be used for analysis -- only using the `include` option for analysis
- [x] `splice-relevant-genes.txt`: list of RNA binding proteins that are annotated to splicing relevant functions from GO.


**NOTE**: For convenience there are two `.tar.gz` archives with the contents described above.

```
gs://robinson-bucket/notebooks/bayesian-modeling/data_bayesian_se_AS_model_B_sex_as_events.tar.gz
gs://robinson-bucket/notebooks/bayesian-modeling/assets_bayesian_se_AS_model_B_sex_as_events.tar.gz
```

Before running the notebook, one can unpack the contents of the archives in the `sbas-nf/data` folder as required by executing the following commands:

```bash
# git clone https://github.com/TheJacksonLaboratory/sbas-nf
tar xvzf data_bayesian_se_AS_model_B_sex_as_events.tar.gz -C data/
tar xvzf assets_bayesian_se_AS_model_B_sex_as_events.tar.gz -C assets/

```


## Loading dependencies

If `conda` is available on your environment you can install the required dependencies by running the following commands:


```bash
time conda install -y r-base==3.6.2 &&
conda install -y r-ggplot2 r-ggsci r-coda r-rstan r-rjags r-compute.es r-snakecase &&
Rscript -e 'install.packages("runjags", repos = "https://cloud.r-project.org/")'
```



In [1]:
# Start the clock!
start_time <- Sys.time()

In [2]:
# dataviz dependencies
library(ggplot2)
library(ggsci)
library(grid)
library(gridExtra)
library(stringr)
library(snakecase)

# BDA2E-utilities dependencies
library(rstan)
library(parallel)
library(rjags)
library(runjags)
library(compute.es)

Warning message:
“package ‘ggplot2’ was built under R version 3.6.3”
Warning message:
“package ‘ggsci’ was built under R version 3.6.3”
Warning message:
“package ‘gridExtra’ was built under R version 3.6.3”
Warning message:
“package ‘snakecase’ was built under R version 3.6.3”
Warning message:
“package ‘rstan’ was built under R version 3.6.3”
Loading required package: StanHeaders

Warning message:
“package ‘StanHeaders’ was built under R version 3.6.3”
rstan (Version 2.19.3, GitRev: 2e1f913d3ca3)

For execution on a local, multicore CPU with excess RAM we recommend calling
options(mc.cores = parallel::detectCores()).
To avoid recompilation of unchanged Stan programs, we recommend calling
rstan_options(auto_write = TRUE)

Warning message:
“package ‘rjags’ was built under R version 3.6.3”
Loading required package: coda

Warning message:
“package ‘coda’ was built under R version 3.6.3”

Attaching package: ‘coda’


The following object is masked from ‘package:rstan’:

    traceplot


Linke

Previously used list of tissues to use for the Hierarchical Bayesian modelling:



```R
tissue.list<-c("Heart - Left Ventricle",
               "Breast - Mammary Tissue",
               "Brain - Cortex.Brain - Frontal Cortex (BA9).Brain - Anterior cingulate cortex (BA24)",
               "Adrenal Gland",
               "Adipose - Subcutaneous",
               "Muscle - Skeletal",
               "Thyroid",
               "Cells - Transformed fibroblasts",
               "Artery - Aorta",
               "Skin - Sun Exposed (Lower leg).Skin - Not Sun Exposed (Suprapubic)")
```

In [3]:
tissues_df <- readr::read_delim("../assets/tissues.tsv", delim = "\t")

Parsed with column specification:
cols(
  name = col_character(),
  female = col_double(),
  male = col_double(),
  include = col_double(),
  display.name = col_character()
)



In [4]:
tissue.list <- tissues_df$name[ tissues_df$include ==1]

In [5]:
message(length(tissue.list), " tissues")
cat(tissue.list, sep = "\n")

39 tissues



adipose_subcutaneous
adipose_visceral_omentum
adrenal_gland
artery_aorta
artery_coronary
artery_tibial
brain_caudate_basal_ganglia
brain_cerebellar_hemisphere
brain_cerebellum
brain_cortex
brain_frontal_cortex_ba_9
brain_hippocampus
brain_hypothalamus
brain_nucleus_accumbens_basal_ganglia
brain_putamen_basal_ganglia
brain_spinal_cord_cervical_c_1
breast_mammary_tissue
cells_cultured_fibroblasts
cells_ebv_transformed_lymphocytes
colon_sigmoid
colon_transverse
esophagus_gastroesophageal_junction
esophagus_mucosa
esophagus_muscularis
heart_atrial_appendage
heart_left_ventricle
liver
lung
muscle_skeletal
nerve_tibial
pancreas
pituitary
skin_not_sun_exposed_suprapubic
skin_sun_exposed_lower_leg
small_intestine_terminal_ileum
spleen
stomach
thyroid
whole_blood


In [6]:
tissue_index = 17
tissue <- tissue.list[tissue_index]  #can be replaced with a loop or argument to choose a different tissue

In [7]:
tissue

[1] "breast_mammary_tissue"

## Pattern for choosing `topTable()` files from `limma`

```bash
# {as_site_type} + '_' + {tissue} + '_' + suffix_pattern 
se_skin_not_sun_exposed_suprapubic_AS_model_B_sex_as_events.csv
```

In [8]:
dataDir <- "../data/"
assetsDir <- "../assets/"
as_site_type <- "se"
suffix_pattern <- "AS_model_B_sex_as_events_refined.csv"

file.with.de.results <- paste0(dataDir, as_site_type, "_", tissue, "_" , suffix_pattern  )
file.with.de.results
file.exists(file.with.de.results)
system( paste0("ls -l ", file.with.de.results), intern = TRUE )

[1] "../data/se_breast_mammary_tissue_AS_model_B_sex_as_events_refined.csv"

[1] TRUE

[1] "-rw-r--r-- 1 jovyan users 393101 Jun 18 16:59 ../data/se_breast_mammary_tissue_AS_model_B_sex_as_events_refined.csv"

In [9]:
events.table         <- read.table(file.with.de.results, sep = ",")
head(events.table, 2)

,logFC,AveExpr,t,P.Value,adj.P.Val,B
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
XIST-10154,-6.796072,1.385720,-38.73465,5.301573e-131,1.523859e-126,280.9110
XIST-10149,-7.124726,1.597306,-38.68238,7.868530e-131,1.523859e-126,280.5225


## Add annotation columns to the topTable dataframe:

The feature information is encoded in the topTable dataframe as rownames. The `ID` and `geneSymbol` variables have been combined in the following pattern:

```console
{geneSymbol}-{ID} 
```

- `ID`: everything **_after_** last occurence of hyphen `-`
example: 
```R
stringr::str_replace("apples - oranges - bananas", "^.+-", "")
```

```console
# output:

' bananas'
```

- `geneSymbol`: everything **_before_** last occurence of `-`
example: 

```R
sub('-[^-]*$', '',"apples - oranges - bananas")
```

```console
# output:

'apples - oranges '
```

```diff
- NOTE: The above solution covers the cases where a hyphen is part of the geneSymbol.
```

In [10]:
cols_initially <- colnames(events.table)
cols_initially

[1] "logFC"     "AveExpr"   "t"         "P.Value"   "adj.P.Val" "B"

In [11]:
events.table[["ID"]] <- stringr::str_replace(rownames(events.table),  "^.+-", "")
events.table[["gene_name"]] <- sub('-[^-]*$', '', rownames(events.table))

In [12]:
keepInOrderCols <- c("gene_name", "ID", cols_initially)

In [13]:
events.table <- events.table[ , keepInOrderCols ]

In [14]:
tail(events.table, 2)

,gene_name,ID,logFC,AveExpr,t,P.Value,adj.P.Val,B
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
TYMP-10179,TYMP,10179,0.6141156,2.719659,2.667809,0.007979548,0.04792555,-3.370464
CALB2-21749,CALB2,21749,0.9789023,2.931131,2.652402,0.008352148,0.04956392,-3.374802


## Define filepaths of required inputs

`file.with.de.results` has been defined above

In [15]:
rbp.table.name        <- paste0(assetsDir, "splice-relevant-genes.txt")
file.exists(rbp.table.name)

[1] TRUE

In [16]:
events.table.name     <- paste0(dataDir, "fromGTF.SE.txt")
file.exists(events.table.name)

[1] TRUE

In [17]:
se.jc.ijc.counts.file.name  <- paste0(dataDir, "rmats_final.se.jc.ijc.txt.gz")
file.exists(se.jc.ijc.counts.file.name)

[1] TRUE

In [18]:
se.jc.sjc.counts.file.name <- paste0(dataDir, "rmats_final.se.jc.sjc.txt.gz")
file.exists(se.jc.sjc.counts.file.name)

[1] TRUE

In [19]:
metadata.file.name    <- paste0(dataDir, "srr_pdata.csv")
file.exists(metadata.file.name)

[1] TRUE

In [20]:
expression.file.name  <- paste0(dataDir, "GTEx_Analysis_2017-06-05_v8_RNASeQCv1.1.9_gene_tpm.gct")
file.exists(expression.file.name)

[1] TRUE

## Use the define filepaths to load/read in the tables 

Load the sjc and sjc count matrices, and the list of RNA binding proteins that are annotated to either:
- mRNA splicing, via spliceosome `(GO:0000398)`,
- regulation of mRNA splicing, via spliceosome `(GO:0048024)`, or 
- both. 

The table has the:
- Gene Symbol
- the Uniprot ID (`uprot.id`)
- the NCBI Gene ID (`gene.id`) and 
- boolean columns for being 
  - `S`=mRNA splicing, via spliceosome `(GO:0000398)` and 
  - `R`=regulation of mRNA splicing, via spliceosome `(GO:0048024)`.

### Filtering of the `topTable()` object

- `abs(events.table$logFC)>=log2(1.5)`
- `events.table$adj.P.Val<=0.05`

In [21]:
dim(events.table)
events.table <- events.table[abs(events.table$logFC)>=log2(1.5) & events.table$adj.P.Val<=0.05,]
dim(events.table)
head(events.table,2)

[1] 3163    8

[1] 3163    8

,gene_name,ID,logFC,AveExpr,t,P.Value,adj.P.Val,B
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
XIST-10154,XIST,10154,-6.796072,1.385720,-38.73465,5.301573e-131,1.523859e-126,280.9110
XIST-10149,XIST,10149,-7.124726,1.597306,-38.68238,7.868530e-131,1.523859e-126,280.5225


Make sure this command has been executed before `gunzip sbas/data/fromGTF.*` as the files are expected uncompressed.


In [22]:
annot.table  <- read.table(events.table.name,header=T)
dim(annot.table)
head(annot.table, 1)

[1] 42611    11

,ID,GeneID,geneSymbol,chr,strand,exonStart_0base,exonEnd,upstreamES,upstreamEE,downstreamES,downstreamEE
,<int>,<fct>,<fct>,<fct>,<fct>,<int>,<int>,<int>,<int>,<int>,<int>
1,1,ENSG00000034152.18,MAP2K3,chr17,+,21287990,21288091,21284709,21284969,21295674,21295769


In [23]:
merged.table <- merge(events.table, annot.table, by="ID")

In [24]:
dim(merged.table)
head(merged.table, 2)

[1] 3163   18

,ID,gene_name,logFC,AveExpr,t,P.Value,adj.P.Val,B,GeneID,geneSymbol,chr,strand,exonStart_0base,exonEnd,upstreamES,upstreamEE,downstreamES,downstreamEE
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>,<fct>,<fct>,<fct>,<int>,<int>,<int>,<int>,<int>,<int>
1,10012,RBM4B,0.8450169,1.183428,6.352709,6.335981e-10,4.076604e-08,12.144677,ENSG00000173914.12,RBM4B,chr11,-,66670935,66670983,66668970,66669291,66676667,66677091
2,10013,RBM4B,-0.7621263,1.752806,-4.689350,3.885523e-06,9.641124e-05,3.795546,ENSG00000173914.12,RBM4B,chr11,-,66668614,66669291,66664997,66665578,66676667,66677091


In [25]:
rbp.table    <- read.table(rbp.table.name,sep="\t",header=TRUE)
dim(rbp.table)
head(rbp.table, 1)

[1] 351   6

,Gene,uprot.id,gene.id,S,R,omim
,<fct>,<fct>,<int>,<lgl>,<lgl>,<fct>
1,AAR2,Q9Y312,25980,TRUE,FALSE,n/a


Make sure this command has been executed before `gunzip sbas/data/rmats_final.se.jc.*jc.*` as the files are expected uncompressed.


In [26]:
se.jc.ijc.counts   <- as.data.frame(data.table::fread(se.jc.ijc.counts.file.name))
dim(se.jc.ijc.counts)
se.jc.ijc.counts[1:2,1:3]

[1] 42611  8674

,ID,SRR1068788,SRR1068808
,<int>,<int>,<int>
1,1,0,0
2,2,26,247


In [27]:
se.jc.sjc.counts  <- as.data.frame(data.table::fread(se.jc.sjc.counts.file.name))
dim(se.jc.sjc.counts)
se.jc.sjc.counts[1:2,1:3]

[1] 42611  8674

,ID,SRR1068788,SRR1068808
,<int>,<int>,<int>
1,1,2,0
2,2,0,0


## Check `dim()` of loaded objects

In [28]:
dim(events.table)
dim(annot.table)
dim(merged.table)
dim(rbp.table)
dim(se.jc.ijc.counts)
dim(se.jc.sjc.counts)

[1] 3163    8

[1] 42611    11

[1] 3163   18

[1] 351   6

[1] 42611  8674

[1] 42611  8674

## Read sample info

In [29]:
metadata.file.name
file.exists(metadata.file.name)
system(paste0("ls -l", " ../data/srr_pdata.csv"), intern = TRUE)

[1] "../data/srr_pdata.csv"

[1] TRUE

[1] "-rw-r--r-- 1 jovyan users 4230047 Jun 17 12:00 ../data/srr_pdata.csv"

In [30]:
meta.data <- readr::read_csv(metadata.file.name)
dim(meta.data)
head(meta.data, 1)

Parsed with column specification:
cols(
  .default = col_double(),
  SAMPID = col_character(),
  SMATSSCR = col_character(),
  SMCENTER = col_character(),
  SMPTHNTS = col_character(),
  SMTS = col_character(),
  SMTSD = col_character(),
  SMUBRID = col_character(),
  SMNABTCH = col_character(),
  SMNABTCHT = col_character(),
  SMNABTCHD = col_character(),
  SMGEBTCH = col_character(),
  SMGEBTCHD = col_character(),
  SMGEBTCHT = col_character(),
  SMAFRZE = col_character(),
  SMGTC = col_logical(),
  SMNUMGPS = col_logical(),
  SM550NRM = col_logical(),
  SM350NRM = col_logical(),
  SMMNCPB = col_logical(),
  SMMNCV = col_logical()
  # ... with 6 more columns
)

See spec(...) for full column specifications.



[1] 7872   68

SAMPID,SMATSSCR,SMCENTER,SMPTHNTS,SMRIN,SMTS,SMTSD,SMUBRID,SMTSISCH,SMTSPAX,⋯,SMRRNART,SME1MPRT,SMNUM5CD,SMDPMPRT,SME2PCTS,SUBJID,SEX,AGE,DTHHRDY,SRR
<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<lgl>,<dbl>,<dbl>,<chr>,<dbl>,<chr>,<dbl>,<chr>
GTEX.PW2O.0006.SM.2I3DV,NA,B1,NA,7.4,Blood,Whole Blood,13756,-126,NA,⋯,0.00351302,0.859573,NA,0,50.6829,GTEX-PW2O,1,20-29,0,SRR604002


In [31]:
meta.data$SMTSD[1:3]

[1] "Whole Blood"            "Lung"                   "Adipose - Subcutaneous"

In [32]:
meta.data[["SMTSD"]] <- as.character(meta.data[["SMTSD"]])

In [33]:
meta.data$SMTSD[1:3]

[1] "Whole Blood"            "Lung"                   "Adipose - Subcutaneous"

In [34]:
meta.data <- meta.data[ snakecase::to_snake_case(meta.data$SMTSD) == tissue,]

In [35]:
tissue
dim(meta.data)
meta.data[1:2,1:8]

[1] "breast_mammary_tissue"

[1] 178  68

SAMPID,SMATSSCR,SMCENTER,SMPTHNTS,SMRIN,SMTS,SMTSD,SMUBRID
<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>
GTEX.S4Q7.1126.SM.4AD6R,0,B1,2 aliquots,6.9,Breast,Breast - Mammary Tissue,8367
GTEX.ZZ64.1226.SM.5E43R,0,B1,2 pieces; fibroadipose tissue without ducts,7.2,Breast,Breast - Mammary Tissue,8367


In [36]:
# Undo snakecase of SMTSD
tissue
tissue <- unique(meta.data$SMTSD)
tissue

[1] "breast_mammary_tissue"

[1] "Breast - Mammary Tissue"

In [37]:
dim(se.jc.ijc.counts)
se.jc.ijc.counts   <- se.jc.ijc.counts[,colnames(se.jc.ijc.counts) %in% meta.data$SRR]
dim(se.jc.ijc.counts)

[1] 42611  8674

[1] 42611   178

In [38]:
dim(se.jc.sjc.counts)
se.jc.sjc.counts  <- se.jc.sjc.counts[,colnames(se.jc.sjc.counts) %in% meta.data$SRR]
dim(se.jc.sjc.counts)

[1] 42611  8674

[1] 42611   178

## This notebook only signficant events are used - filtering now on Adj.P Val

If there are more than 100 events that are significant, reduce this by ordering on the adjusted p-value (column Adj.P.Val).  

Then update the ijc and sjc matrices and the merged.table (annotations)


In [39]:
if (nrow(se.jc.sjc.counts) > 100 ) {
    head(merged.table,2)
    dim(merged.table)
    order_by_adj.P.Val <- order(merged.table$adj.P.Val,decreasing=TRUE)
    head(merged.table[order_by_adj.P.Val,],2)
    keep_IDs <- merged.table[order_by_adj.P.Val,]$ID[1:100]
    head(keep_IDs)
    keep_IDs <- keep_IDs[order(keep_IDs, decreasing=FALSE)]
    head(keep_IDs)
    # counts table will be identical in rows
    counts.keep <- as.character(rownames(se.jc.sjc.counts)) %in% as.character(keep_IDs)
    table(counts.keep)
    # not so sure about merged.table - so lets have a different logical key
    merged.keep <- as.character(merged.table$ID) %in% as.character(keep_IDs)
    table(merged.keep)
    
    # update the sjc counts 
    dim(se.jc.sjc.counts)
    se.jc.sjc.counts  <- se.jc.sjc.counts[counts.keep,]
    dim(se.jc.sjc.counts)
 
    # update the ijc counts
    dim(se.jc.ijc.counts)
    se.jc.ijc.counts   <- se.jc.ijc.counts[counts.keep,]
    dim(se.jc.ijc.counts)
    
    # merged table
    dim(merged.table)
    merged.table       <- merged.table[merged.keep,] 
    dim(merged.table)
}

[1] 100  18

In [40]:
dim(se.jc.ijc.counts)
dim(se.jc.sjc.counts)
dim(merged.table)

[1] 100 178

[1] 100 178

[1] 100  18

## Read expression data:

In [ ]:
expression.file.name
file.exists(expression.file.name)

In [ ]:
expression.mat <- read.table(expression.file.name, 
                             nrows = 1,
                             sep = "\t",
                             header = T,
                             skip = 2)

In [ ]:
dim(expression.mat)
expression.mat[1:2,1:4]

In [ ]:
colnames(expression.mat)[1:3]

In [ ]:
colnames.expression.mat <- colnames(expression.mat)

In [ ]:
length(colnames.expression.mat)
colnames.expression.mat[1:4]

In [ ]:
length(colnames.expression.mat)

In [ ]:
total.samples           <- length(colnames.expression.mat)
total.samples

In [ ]:
meta.data$SAMPID[1]
gsub("-","\\.",meta.data$SAMPID[1])

In [ ]:
meta.data$SAMPID   <- gsub("-","\\.",meta.data$SAMPID)

In [ ]:
dim(meta.data)
meta.data               <- meta.data[meta.data$SAMPID %in% colnames(expression.mat),]
dim(meta.data)

In [ ]:
expression.mat[1:2,1:4]

In [ ]:
meta.data <- meta.data[!duplicated(meta.data$SAMPID),]

In [ ]:
dim(meta.data)

In [ ]:
se.jc.ijc.counts <- se.jc.ijc.counts[,colnames(se.jc.ijc.counts) %in% meta.data$SRR]
dim(se.jc.ijc.counts)
se.jc.ijc.counts[1:2,1:4]

In [ ]:
se.jc.sjc.counts <- se.jc.sjc.counts[,colnames(se.jc.sjc.counts) %in% meta.data$SRR]
dim(se.jc.sjc.counts)
se.jc.sjc.counts[1:2,1:4]

In [ ]:
meta.data <- meta.data[meta.data$SRR %in% colnames(se.jc.ijc.counts),]
dim(meta.data)
meta.data[1:2,1:8]

In [ ]:
colnames.expression.mat[1:4]

In [ ]:
dim(expression.mat)
expression.mat[1:2,1:4]

In [ ]:
tissue <- unique(meta.data$SMTSD [ meta.data$SMTSD == tissue])
tissue

In [ ]:
col.in.tissue<-c()
for (col in colnames.expression.mat)
  
  col.in.tissue<-c(col.in.tissue, (col %in% meta.data$SAMPID) && (meta.data$SMTSD[which(meta.data$SAMPID==col)] %in% tissue) && (meta.data$SUBJID[which(meta.data$SAMPID==col)]!='GTEX-11ILO'))

In [ ]:
length(col.in.tissue)
table(col.in.tissue)

In [ ]:
length(colnames.expression.mat)
length(col.in.tissue)

col.in.tissue[1:3]

In [ ]:
# colClasses is used to skip columns
expression.mat <-read.table(expression.file.name, 
                            sep= "\t",
                            header = T,
                            skip = 2, 
                            colClasses = ifelse(col.in.tissue, "numeric", "NULL"))

In [ ]:
length(col.in.tissue)

## Read gene names:

In [ ]:
dim(expression.mat)
expression.mat <- expression.mat[,order(match(colnames(expression.mat),meta.data$SAMPID))]
dim(expression.mat)

In [ ]:
se.jc.ijc.counts     <- se.jc.ijc.counts[,order(match(colnames(se.jc.ijc.counts),meta.data$SRR))]
dim(se.jc.ijc.counts)

In [ ]:
se.jc.sjc.counts    <- se.jc.sjc.counts[,order(match(colnames(se.jc.sjc.counts),meta.data$SRR))]
dim(se.jc.sjc.counts)

In [ ]:
all.genes      <- read.table(expression.file.name,sep="\t",header=T,skip=2,colClasses = c(rep("character", 2), rep("NULL", total.samples-2)))
dim(all.genes)
head(all.genes, 2)

In [ ]:
expression.mat <- expression.mat[!duplicated(all.genes$Description),]
dim(expression.mat)
expression.mat[1:2,1:4]

In [ ]:
all.genes      <- all.genes[!duplicated(all.genes$Description),]
dim(all.genes)

In [ ]:
se.jc.sjc.counts    <- se.jc.sjc.counts[merged.table$geneSymbol %in% all.genes$Description,]
dim(se.jc.sjc.counts)

In [ ]:
se.jc.ijc.counts     <- se.jc.ijc.counts[merged.table$geneSymbol %in% all.genes$Description,]
dim(se.jc.ijc.counts)

In [ ]:
merged.table   <- merged.table[merged.table$geneSymbol %in% all.genes$Description,]
dim(merged.table)

In [ ]:
gene.names     <- unique(merged.table$geneSymbol)
length(gene.names)

In [ ]:
expression.mat <- expression.mat[all.genes$Description %in% c(as.character(rbp.table$Gene),as.character(gene.names)),]
dim(expression.mat)

In [ ]:
rownames.expression.mat <-all.genes$Description[all.genes$Description %in% c(as.character(rbp.table$Gene),as.character(gene.names))]
length(rownames.expression.mat)

In [ ]:
expression.mat <-expression.mat[!duplicated(rownames.expression.mat),]
dim(expression.mat)

In [ ]:
rownames.expression.mat <-rownames.expression.mat[!duplicated(rownames.expression.mat)]
length(rownames.expression.mat)

## Prepare expression of genes and RBPS:

In [ ]:
num.events     <- nrow(merged.table)
num.events

In [ ]:
event.to.gene  <- c()

In [ ]:
gexp           <- expression.mat[rownames.expression.mat %in% gene.names,]
dim(gexp)

In [ ]:
rownames(gexp) <- rownames.expression.mat[rownames.expression.mat %in% gene.names]

In [ ]:
gexp           <- gexp[order(match(rownames(gexp),gene.names)),]
dim(gexp)
gexp[1:2,1:4]

In [ ]:
gexp           <- log2(gexp+0.5)

In [ ]:
gexp           <- gexp-rowMeans(gexp)

In [ ]:
gexp[apply(gexp,1,sd)>0,] <- gexp[apply(gexp,1,sd)>0,]/apply(gexp[apply(gexp,1,sd)>0,],1,sd)

In [ ]:
rexp           <- expression.mat[rownames.expression.mat %in% rbp.table$Gene,]

In [ ]:
rownames(rexp) <- rownames.expression.mat[rownames.expression.mat %in% rbp.table$Gene]

In [ ]:
rexp           <- rexp[order(match(rownames(rexp),rbp.table$Gene)),]

In [ ]:
rexp           <- log2(rexp+0.5)

In [ ]:
rexp           <- rexp-rowMeans(rexp)

In [ ]:
rexp           <- rexp/apply(rexp,1,function(v){ifelse(sum(v==v[1])<length(v),sd(v),1)})

In [ ]:
for (i in (1:num.events))
  event.to.gene<-c(event.to.gene,which(unique(merged.table$geneSymbol)==merged.table[i,"geneSymbol"]))
    sex<-ifelse(meta.data$SEX==1,1,0)

In [ ]:
sex[1:4]
table(sex)

In [ ]:
end_time <- Sys.time()
end_time - start_time

## Run stan:

In [ ]:
dataList = list(
  as = round(se.jc.sjc.counts) ,   #sjc event counts across experiments
  c = round(se.jc.sjc.counts+se.jc.ijc.counts)    , #total counts for event, i.e. sjc & ijc, across experiments
  gexp = gexp, #read counts for genes (from gtex, take the raw counts) across experiments
  rexp = rexp, #read counts for RBPs (from gtex, take the raw counts)
  event_to_gene = event.to.gene,  #the gene index for each event (1 to the number of distinct genes) 
  Nrbp = nrow(rexp), #number of RBPs
  Nevents = nrow(merged.table),  #most varying AS events in 
  Nexp = ncol(expression.mat),#number of experiments such that we measured each event, gene and RBP in each experiment
  Ngenes = nrow(gexp),
  sex=sex
)


modelString = "
data {
int<lower=0> Nevents;
int<lower=0> Nexp;
int<lower=0> Nrbp;
int<lower=0> Ngenes;
int<lower=0> as[Nevents,Nexp] ;
int<lower=0> c[Nevents,Nexp] ;
matrix[Ngenes,Nexp] gexp ; 
matrix[Nrbp,Nexp] rexp ; 
int<lower=0> event_to_gene[Nevents];
int<lower=0,upper=1> sex[Nexp];

}


parameters {
real beta0[Nevents] ;
real beta1[Nevents] ;
matrix[Nevents,Nrbp] beta2 ;
real beta3[Nevents];
real beta4[Nrbp];

}
model {

for ( i in 1:Nexp ) {  


    for ( j in 1:Nevents ) if (c[j,i]>0) { 

      as[j,i] ~ binomial(c[j,i], inv_logit(beta0[j]+beta1[j]*sex[i]+dot_product(beta2[j,],rexp[,i])+beta3[j]*gexp[event_to_gene[j],i] ) );

  }
}

for (k in 1:Nrbp){

  for ( j in 1:Nevents ) { 

        beta2[j,k] ~normal(beta4[k],1);
  }

  beta4[k]~normal(0,1);

}


for ( j in 1:Nevents ) { 

    beta1[j] ~ normal(0,1);
    beta0[j] ~ normal(0,1);
    beta3[j] ~ normal(0,1);
  }

}
"

# Start the clock!
start_time <- Sys.time()

stanDso <- rstan::stan_model( model_code=modelString ) 
stanFit <- sampling( object=stanDso , 
                    data = dataList , 
                    chains = 3 , 
                    iter = 8000, 
                    warmup = 6000,
                    thin = 1,
                    init = 0, 
                    cores = 3 )

mcmcCoda = coda::mcmc.list( lapply( 1:ncol(stanFit) , function(x) { mcmc(as.array(stanFit)[,x,]) } ) )

end_time <- Sys.time()
end_time - start_time

## Save R objects

In [ ]:
save.image(file = "notebook.RData")
file.exists("notebook.RData")
system("pwd && ls -l notebook.RData", intern = TRUE)

## Metadata

For replicability and reproducibility purposes, we also print the following metadata:

1. Checksums of **"artefacts"**, files generated during the analysis and stored in the folder directory **`data`**
2. List of environment metadata, dependencies, versions of libraries using `utils::sessionInfo()` and [`devtools::session_info()`](https://devtools.r-lib.org/reference/session_info.html)

### 1. Checksums with the sha256 algorithm

In [ ]:
figure_id       <- "bayesian-modeling"

message("Generating sha256 checksums of the artefacts in the `..data/` directory .. ")
system(paste0("cd ../data/ && find . -type f -exec sha256sum {} \\; > ../metadata/",  figure_id, "_sha256sums.txt"), intern = TRUE)
message("Done!\n")

data.table::fread(paste0("../metadata/", figure_id, "_sha256sums.txt"), header = FALSE, col.names = c("sha256sum", "file"))

### 2. Libraries metadata

In [ ]:
dev_session_info   <- devtools::session_info()
utils_session_info <- utils::sessionInfo()

message("Saving `devtools::session_info()` objects in ../metadata/devtools_session_info.rds  ..")
saveRDS(dev_session_info, file = paste0("../metadata/", figure_id, "_devtools_session_info.rds"))
message("Done!\n")

message("Saving `utils::sessionInfo()` objects in ../metadata/utils_session_info.rds  ..")
saveRDS(utils_session_info, file = paste0("../metadata/", figure_id ,"_utils_info.rds"))
message("Done!\n")

dev_session_info$platform
dev_session_info$packages[dev_session_info$packages$attached==TRUE, ]